In [1]:
import pickle
from nltk.corpus import stopwords
import re 
import math
from nltk import word_tokenize
from nltk.stem import PorterStemmer
ps=PorterStemmer()
import numpy as np

In [2]:
#loading pickle files
file1 = open('df.pkl','rb')
df_dict = pickle.load(file1)

file2 = open('file_idx.pkl','rb')
file_lst = pickle.load(file2)

file3 = open('doc_norm.pkl','rb')
l2_norm_dict = pickle.load(file3)

file4 = open('doc_words.pkl','rb')
tot_tkns = pickle.load(file4)

file5 = open('doc_len.pkl','rb')
doc_len = pickle.load(file5)

file6 = open('posting_list.pkl','rb')
tf_dict = pickle.load(file6)


In [3]:
#set of stopwords
stpwrds = set(stopwords.words('english'))

In [4]:
#input
query=input("Enter the query : ")
regex = re.compile('[^a-zA-Z\s]')
words = re.sub(regex,'',query)

#tokenizing and stemming the query
tokens = [word.lower() for word in word_tokenize(words)]
stmd_tkns = [ps.stem(token) for token in tokens if token not in stpwrds]
stmd_tkns

Enter the query : javascript


['javascript']

In [5]:
tf_idf_qry = []
l2_norm = 0

for i in stmd_tkns:
    tf_idf = 0
    if i in tf_dict:
        tf_idf = stmd_tkns.count(i) * math.log(len(file_lst)/df_dict[i])
    tf_idf_qry.append(tf_idf)
    l2_norm += tf_idf ** 2
l2_norm = math.sqrt(l2_norm)

tf_idf_qry = np.array(tf_idf_qry)/l2_norm
    

In [6]:
tf_idf_qry

array([1.])

In [7]:
#calculating the score for each document for a given query
score={}
for i in range(len(file_lst)):
    tf_idf_doc = []
    for t in stmd_tkns:
        c_tf = 0
        if t in tf_dict and i in tf_dict[t]:
            c_tf = (tf_dict[t][i]) * math.log(len(file_lst)/df_dict[t])
        tf_idf_doc.append(c_tf)
    tf_idf_doc = np.array(tf_idf_doc)/l2_norm_dict[i]
    score[i] = np.dot(tf_idf_qry,tf_idf_doc)

In [8]:
#sort similarity score in descending order 
score1=sorted(score.items(),key=lambda x:x[1],reverse=True) 

In [9]:
#printing the top 10 documents
count = 10
for i in score1:
    if count == 0:
        break
    print(file_lst[i[0]],i[1])
    count-=1

C00001.txt 0.7202624698475235
C00580.txt 0.4665687494351361
C00014.txt 0.24137609156149947
C00087.txt 0.2247237595689026
C00660.txt 0.15870216117548389
C00561.txt 0.14000562413956574
C00022.txt 0.13776739681423733
C00585.txt 0.10882667855631256
C00971.txt 0.08847696077232294
C00753.txt 0.07746500282683663
